## Clean schools (i.e. X law school is the same as X university)

In [1]:
import pandas as pd
import numpy as np
import re

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# Manually change two schoosl (University of Washington and Eveleth Junior College) in the 
# school1_alt column that didn't split on its own  
# and reimport
df = pd.read_csv('02-split.csv', na_values = 'Read law')
# Loading in a raw version for double checking as I go
df_raw = pd.read_csv('02-split.csv', na_values = 'Read law')

In [3]:
df = df.replace('Read law', '', regex=True)

In [4]:
# Test regex to keep everything BUT the degree name
# -- this solution works with string extract, but would NaN schools that had no degree
# at the end 

# string_test = "Bowdoin College, A.B."
# print(re.findall('(.*)(?:[,] \w[.]\w[.])', string_test))

# Update regex to work with LL.B., Ph.D., etc. 
# string_test = "Harvard Law School, LL.B."

# print(re.findall('(.*)(?:[,] \w{1,2}[.]\w[.])', string_test))

# df['school1'] = df['school1_alt_raw'].str.extract(r'(.*)(?:[,] \w{1,2}[.]\w[.])')

In [5]:
# # The many iterations of this regex to REMOVE just the degree name
# string_test = "Harvard Law School, LL.B."
# print(re.findall(', \w{1,2}[.]\w[.]', string_test))

# string_test = "University of Santa Clara (now Santa Clara University), B.Sc."
# print(re.findall(', \w{1,2}[.]\w{1,2}[.]', string_test))

# string_test = "Southwestern UniversityA.B."
# print(re.findall(',{0,1}\s{0,1}[A-Z]{1,2}[.]\w{1,2}[.]$', string_test))

# string_test = "SUniversity of California, Berkeley, B.Litt."
# print(re.findall(',{0,1}\s{0,1}[A-Z]{1,2}[.]\w+[.]$', string_test))

# # 0 or 1 commas / 0 or 1 spaces / 1 or 2 capital letters / . / more than one letter / . / end of string

In [6]:
# Replace Brandeis, WashU and any other school with periods in it
# Using hashes in St. so I can add it back later
df = df.replace('St[.]','St###', regex=True)

# Ohio Northern University, Warren G. Harding College of Law
df = df.replace('Ohio Northern University, Warren G. Harding College of Law','Ohio Northern University', regex=True)

# University of Florida College of Law (now Fredric G. Levin College of Law)
df = df.replace('University of Florida College.*','University of Florida', regex=True)

# University of Arizona College of Law (now James E. Rogers College of Law), LL.B.
df = df.replace('University of Arizona College of Law.*','University of Arizona', regex=True)

# Louisiana State University, Paul M. Hebert Law Center, J.D.
df = df.replace('Louisiana State University, Paul.*','Louisiana State University', regex=True)

# University of Louisville Law Department (now Louis D. Brandeis School of Law), LL.B.
df = df.replace('University of Louisville Law.*','University of Louisville', regex=True)

# Ohio State University College of Law (now Michael E. Moritz College of Law), LL.B.
df = df.replace('Ohio State University College of Law.*','Ohio State University', regex=True)

# Temple University School of Law (now James E. Beasley School of Law)
df = df.replace('Temple University School of Law.*','Temple University', regex=True)

In [7]:
# Finally ended up with this one, which captures and removes all the degrees

# Commented out my testing
# string_test = "Harvard Law School, LL.B."
# string_test = "University of Santa Clara (now Santa Clara University), B.Sc."
# string_test = "Southwestern UniversityA.B."
# string_test = "University of California, Berkeley, B.Litt."
# string_test = "Columbia University, Ph.G."
# string_test = "Michigan State University, B.S.E.E."

# print(re.findall(',{0,1}\s{0,1}[A-Z]+[a-z]*[.].*$', string_test))
# 0 or 1 commas + 0 or 1 spaces + 1 or more capital letter + 0 or more lowercase letters + . + anything else, end of string

df = df.replace(',{0,1}\s{0,1}[A-Z]+[a-z]*[.].*$','', regex=True)

In [8]:
# df.school1_alt.unique()

In [9]:
# Replace all schools with the schools in the (now TK)

# string_test = 'Southwest Missouri State University (now Missouri State University)'
# But some schools only have an open parenthese and not a close
# string_test = 'Indiana Law School (now Indiana University Mc'

# OLD
# print(re.findall('(.*)(?: [(]now )', string_test))
# print(re.findall('(.*)(?: [(]now.*[)])', string_test))
# print(re.findall('(?:now )(.*)(?:[)])', string_test))
# print(re.findall('(?:now )(.*)(?:[)]*)', string_test))

# NEW
# print(re.findall('^.*[(]now ', string_test))

df = df.replace('^.*[(]now ','', regex=True)

In [10]:
# Remove trailing close parentheses

# string_test = 'Pritzker)'
# print(re.findall('[)]$', string_test))

df = df.replace('[)]$','', regex=True)

In [11]:
# Replace anything that says 'School of Law' at the end of the school
df = df.replace(' School of Law','', regex=True)
df = df.replace(' College of Law','', regex=True)

In [12]:
# Bring back the periods in St.
df = df.replace('###','.', regex=True)

In [13]:
# Sub this with NaN because it's a scholarship
df = df.replace("Temple Bar Scholar, American Inns of Court", np.nan)

In [14]:
# Changing things that got chopped & renaming law schools to their parent universities
# South Texas College of Law
df = df.replace("South Texas.*", "South Texas College of Law", regex=True)

# Chicago-Kent College of Law
df = df.replace("Chicago-Kent.*", "Chicago-Kent College of Law", regex=True)

# Kinney > McKinney School of Law
df = df.replace(".*Kinney.*", "Indiana University", regex=True)

# Cumberland > should actually be part of Samford University
df = df.replace(".*Cumberland.*", "Samford University", regex=True)

# Benton
df = df.replace("Benton", "Benton College of Law")

# Sturm > University of Denver College of Law (now Sturm College of Law)
df = df.replace("Sturm", "University of Denver")

# Sandra Day O’Connor > ASU
df = df.replace("Sandra Day O’Connor", "Arizona State University")

# Pritzker
df = df.replace(".*Pritzker.*", "Northwestern University", regex=True)

# Carey Law School
# Francis King University of Maryland
df = df.replace("Carey.*", "University of Maryland", regex=True)
df = df.replace(".*University of Maryland.*", "University of Maryland", regex=True)

# Maurer
df = df.replace("Maurer.*", "Indiana University", regex=True)

# Mitchell Hamline
df = df.replace("Mitchell Hamline.*", "Mitchell Hamline School of Law", regex=True)

# Tacoma Law > Seattle University School of Law?
# df = df.replace("Tacoma Law School.*", "University of Washington", regex=True)

df = df.replace("Rutgers.*", "Rutgers University", regex=True)

# Penn State Dickinson Law
df = df.replace("Penn State.*", "Pennsylvania State University", regex=True)

# UIC John Marshall Law School
df = df.replace("UIC John Marshall Law School", "University of Illinois Chicago", regex=True)

# George Mason
df = df.replace("Antonin Scalia Law School", "George Mason University", regex=True)

# University of Florida Fredric
df = df.replace("University of Florida.*", "University of Florida", regex=True)

# Indiana University Maurer / Maurer
df = df.replace("Indiana University.*", "Indiana University", regex=True)

# Washington University (was Washington University School of Law) > Washington University in St. Louis
df = df.replace(".*Washington University.*", "Washington University in St. Louis", regex=True)


In [15]:
# Cleaning, converting college > university 
df = df.replace("Harvard.*", "Harvard University", regex=True)
df = df.replace("Yale.*", "Yale University", regex=True)
df = df.replace("Washburn.*", "Washburn University", regex=True)
df = df.replace("Wake Forest.*", "Wake Forest University", regex=True)
df = df.replace("Adams State.*", "Adams State University", regex=True)
df = df.replace("Tufts.*", "Tufts University", regex=True)
df = df.replace("Plymouth State.*", "Plymouth State University", regex=True)
df = df.replace("University of Kansas.*", "University of Kansas", regex=True)
df = df.replace("Elmhurst.*", "Elmhurst University", regex=True)
df = df.replace("Lake Forest.*", "Lake Forest College", regex=True)
df = df.replace("Otterbein.*", "Otterbein University", regex=True)
df = df.replace("San Diego State.*", "San Diego State University", regex=True)
df = df.replace("Virginia State.*", "Virginia State University", regex=True)
df = df.replace("Dordt.*", "Dordt University", regex=True)
df = df.replace("Fordham.*", "Fordham University", regex=True)
df = df.replace("Emory.*", "Emory University", regex=True)

In [16]:
# Renaming more schools

# University of Pennsylvania, Wharton School
df = df.replace("University of Pennsylvania.*", "University of Pennsylvania", regex=True)

# Columbia College and Columbia Law School 
# had to make regex here more specific so at to not replace District of Columbia
df = df.replace("Columbia\s[A-Z].*", "Columbia University", regex=True)

# University of Michigan Law School
df = df.replace("University of Michigan.*", "University of Michigan", regex=True)

# Tulane University Law School
df = df.replace("Tulane University.*", "Tulane University", regex=True)

# Louisiana State University Law School
df = df.replace("Louisiana State University.*", "Louisiana State University", regex=True)

# George Washington University Law School
df = df.replace("George Washington University.*", "George Washington University", regex=True)

# Notre Dame Law School / Notre Dame University
df = df.replace(".*Notre Dame.*", "University of Notre Dame", regex=True)

# Western Michigan University, Honors College
df = df.replace("Western Michigan University.*", "Western Michigan University", regex=True)

# Old University of Chicago / University of Chicago Law School
df = df.replace(".*University of Chicago.*", "University of Chicago", regex=True)

# Illinois Wesleyan University Law School
df = df.replace("Illinois Wesleyan University.*", "Illinois Wesleyan University", regex=True)

# University of Minnesota Law School
df = df.replace("University of Minnesota.*", "University of Minnesota", regex=True)

# University of Southern California Law School
df = df.replace("University of Southern California.*", "University of Southern California", regex=True)

# Cornell Law School / Cornell University Department of Law
df = df.replace(".*Cornell.*", "Cornell University", regex=True)

# State University of New York at Buffalo Law School
df = df.replace("State University of New York at Buffalo.*", "State University of New York at Buffalo", regex=True)

# Stanford Graduate School of Business
df = df.replace("Stanford.*", "Stanford University", regex=True)

# University of Wisconsin Law School
df = df.replace("University of Wisconsin.*", "University of Wisconsin", regex=True)

# Vanderbilt University Law School
df = df.replace("Vanderbilt.*", "Vanderbilt University", regex=True)

# University of Colorado Law School
df = df.replace(".*Colorado.*", "University of Colorado", regex=True)

# Marquette University Law School
df = df.replace("Marquette.*", "Marquette University", regex=True)

# University of California, Berkeley, School of Jurisprudence
df = df.replace("University of California, Berkeley.*", "University of California, Berkeley", regex=True)

# University of Cambridge Faculty of Law / University of Cambridge, Queens' College
df = df.replace("University of Cambridge.*", "University of Cambridge", regex=True)

# Suffolk University Law School
df = df.replace("Suffolk.*", "Suffolk University", regex=True)

# Wayne State University Law School
df = df.replace("Wayne State University.*", "Wayne State University", regex=True)

# William and Mary Law School
df = df.replace("William and Mary Law School", "College of William and Mary", regex=True)

# College of Law of Central University
df = df.replace("College of Law of Central University", "Central University", regex=True)

# University of Oxford, University College
df = df.replace("University of Oxford.*", "University of Oxford", regex=True)

# Drake University Law School
df = df.replace("Drake.*", "Drake University", regex=True)

# Hamilton College Law Department
df = df.replace("Hamilton College.*", "Hamilton College", regex=True)

# University of Houston Law Center
df = df.replace("University of Houston.*", "University of Houston", regex=True)

# University of Oxford, Balliol College, Rhodes Scholar
df = df.replace("University of Oxford.*", "University of Oxford", regex=True)

# State University of New York at Albany, Certificate in Regulatory Economics
df = df.replace("State University of New York at Albany.*", "State University of New York at Albany", regex=True)

# University of California, Los Angeles,
df = df.replace("University of California, Los Angeles,.*", "University of California, Los Angeles", regex=True)

# Georgetown University Law Center
df = df.replace("Georgetown.*", "Georgetown University", regex=True)

# University of the Pacific, McGeorge
df = df.replace("University of the Pacific.*", "University of the Pacific", regex=True)

# Drexel Institute of Art, Science and Industry
df = df.replace("Drexel.*", "Drexel University", regex=True)

# University of Louisville Louis
df = df.replace("University of Louisville.*", "University of Louisville", regex=True)

# Central University (now combined with another institution to form Eastern Kentucky University)
df = df.replace("Central University", "Central University (now Eastern Kentucky University)", regex=True)


In [17]:
# More cleaning, this time for merging into the addresses dataframe in step 5
df = df.replace("St. Petersburg Junior College", "St. Petersburg College", regex=True)
df = df.replace("Trinity College, Hartford, Connecticut", "Trinity College")
df = df.replace("St. John's College, Annapolis, Maryland", "St. John's College")
df = df.replace("Miami University, Ohio", "Miami University")
df = df.replace("West Virginia State College", "West Virginia State University")
df = df.replace("American University, Washington", "American University")
df = df.replace("Brigham Young University -- Idaho", "Brigham Young University - Idaho")
df = df.replace("Union University, Jackson, Tennessee", "Union University")
df = df.replace("University of Missouri-Kansas City", "University of Missouri - Kansas City")
df = df.replace("Southern University", "Southern University and A & M College")

In [18]:
# Clean straggling parentheses and punctuation

# De.Paul University
df = df.replace("De[.]Paul.*", "DePaul University", regex=True)

# University of Utah (now
df = df.replace("University of Utah.*", "University of Utah", regex=True)

# Washington and Lee University)
df = df.replace("Washington and Lee University.*", "Washington and Lee University", regex=True)

# University of Memphis), Humphreys
df = df.replace("University of Memphis.*", "University of Memphis", regex=True)

# Michigan State University), Certificate of Completion
df = df.replace("Michigan State University.*", "Michigan State University", regex=True)

# Case Western Reserve University)
df = df.replace("Case Western Reserve University.*", "Case Western Reserve University", regex=True)

In [19]:
df.to_csv('03-clean.csv', index=False)